<a href="https://colab.research.google.com/github/hemantam1/WebScraping/blob/main/PDC_TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title Mount with drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#@title Include all dependencies
import os
import cv2
from PIL import Image
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import load_model
from tensorflow.keras.utils import img_to_array
#from keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
# Try with generalized method for importing number of classes
import pathlib
import numpy as np
data_dir = pathlib.Path('/content/drive/MyDrive/IC_CombineDB/Image_Classification/dataset/skin_plant_disease/train/')
train_dir = data_dir
valid_dir = pathlib.Path('/content/drive/MyDrive/IC_CombineDB/Image_Classification/dataset/skin_plant_disease/validation/')
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
CLASS_NAMES

array(['malignant', 'benign', 'Anthracnose', 'nutrient_deficiency',
       'Alternaria_Alternata', 'Bacterial_Blight', 'Cercospora_Leaf_Spot',
       'Healthy_Leaves'], dtype='<U20')

In [5]:
import random
all_images = list(data_dir.glob('*/*'))
all_images = [str(path) for path in all_images]
random.shuffle(all_images)
data_size=len(all_images)
print(data_size)

3972


In [ ]:
num_classes=len(CLASS_NAMES)
# importing the libraries
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications import VGG16
#from keras.preprocessing import image

IMAGE_SIZE = [150, 150]  # we will keep the image size as (64,64). You can increase the size for better results. 

# loading the weights of VGG16 without the top layer. These weights are trained on Imagenet dataset.
vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  # input_shape = (64,64,3) as required by VGG

# this will exclude the initial layers from training phase as there are already been trained.
for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
#x = Dense(128, activation = 'relu')(x)   # we can add a new fully connected layer but it will increase the execution time.
x = Dense(num_classes, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.

model = Model(inputs = vgg.input, outputs = x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])